# Data Wrangling

In [1]:
import pandas as pd
import requests

In [2]:
# lifehacks
lhs_url = "https://api.pushshift.io/reddit/search/submission/?subreddit=lifehacks&metadata=true&size=0"
lht_url = "https://api.pushshift.io/reddit/search/submission/?subreddit=lifehacks&metadata=true&size=0&is_self=true"

# lifeprotips
lpts_url = "https://api.pushshift.io/reddit/search/submission/?subreddit=LifeProTips&metadata=true&size=0"
lptt_url = "https://api.pushshift.io/reddit/search/submission/?subreddit=LifeProTips&metadata=true&size=0&is_self=true"

In [3]:
res_lhs = requests.get(lhs_url)
res_lht = requests.get(lht_url)
res_lpts = requests.get(lpts_url)
res_lptt = requests.get(lptt_url)

print(res_lhs.status_code, res_lht.status_code)
print(res_lpts.status_code, res_lptt.status_code)

200 200
200 200


All requests successful.

In [4]:
lhs_count = res_lhs.json()['metadata']['total_results']
lht_count = res_lht.json()['metadata']['total_results']

lpts_count = res_lpts.json()['metadata']['total_results']
lptt_count = res_lptt.json()['metadata']['total_results']

print(f"LifeHacks Total Submissions: {lhs_count}\nLifeHacks Total Self-Text Posts: {lht_count}")
print(f"LifeProTips Total Submissions: {lpts_count}\nLifeProTips Total Self-Text Posts: {lptt_count}")

LifeHacks Total Submissions: 81718
LifeHacks Total Self-Text Posts: 23233
LifeProTips Total Submissions: 555135
LifeProTips Total Self-Text Posts: 534133
